# Design Patterns with First-Class Functions

## Case Study: Refactoring Strategy

### Classic Strategy

In [2]:
from abc import ABC, abstractmethod
from collections.abc import Sequence
from decimal import Decimal
from typing import NamedTuple, Optional


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self) -> Decimal:
        return self.price * self.quantity


class Order(NamedTuple):  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional['Promotion'] = None

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))

    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}>'


class Promotion(ABC):  # the Strategy: on abstract base class
    @abstractmethod
    def discount(self, order: Order) -> Decimal:
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""

    def discount(self, order: Order) -> Decimal:
        rate = Decimal('0.05')
        if order.customer.fidelity >= 1000:
            return order.total() * rate
        return Decimal(0)


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""

    def discount(self, order: Order) -> Decimal:
        discount = Decimal(0)
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * Decimal('0.1')
        return discount


class LargeOrderPromo(Promotion):
    """7% discount for orders with 10 or more distinct items"""

    def discount(self, order: Order) -> Decimal:
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * Decimal('0.07')
        return Decimal(0)

In [3]:
joe = Customer('John Doe', 0)  #1
ann = Customer('Ann Smith', 1100)
cart = (  #2
    LineItem('banana', 4, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5))
)
Order(joe, cart, FidelityPromo())  #3

<Order total: 42.00 due: 42.00>

In [4]:
Order(ann, cart, FidelityPromo())  #4

<Order total: 42.00 due: 39.90>

In [5]:
banana_cart = (
    LineItem('banana', 30, Decimal('.5')),  #5
    LineItem('apple', 10, Decimal('1.5'))
)
Order(joe, banana_cart, BulkItemPromo())  #6

<Order total: 30.00 due: 28.50>

In [7]:
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))  #7
Order(joe, long_cart, LargeOrderPromo())  #8

<Order total: 10.00 due: 9.30>

In [8]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

### Function-Oriented Strategy

In [12]:
from collections.abc import Sequence
from dataclasses import dataclass
from decimal import Decimal
from typing import Optional, Callable, NamedTuple


class Customer(NamedTuple):
    name: str
    fidelity: int


class LineItem(NamedTuple):
    product: str
    quantity: int
    price: Decimal

    def total(self):
        return self.price * self.quantity


@dataclass(frozen=True)
class Order:  # the Context
    customer: Customer
    cart: Sequence[LineItem]
    promotion: Optional[Callable[['Order'], Decimal]] = None  #1

    def total(self) -> Decimal:
        totals = (item.total() for item in self.cart)
        return sum(totals, start=Decimal(0))
    
    def due(self) -> Decimal:
        if self.promotion is None:
            discount = Decimal(0)
        else:
            discount = self.promotion(self)  #2
        return self.total() - discount
    
    def __repr__(self):
        return f'<Order total: {self.total():.2f} due: {self.due():.2f}'  #3


def fidelity_promo(order: Order) -> Decimal:  #4
    """5% discount for customer with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)


def bulk_item_promo(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount


def large_order_promo(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)

In [3]:
joe = Customer('John Doe', 0)  #1
ann = Customer('Ann Smith', 1100)
cart = (  #2
    LineItem('banana', 4, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5')),
    LineItem('watermelon', 5, Decimal(5))
)
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00

In [4]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90

In [5]:
banana_cart = (
    LineItem('banana', 30, Decimal('.5')),
    LineItem('apple', 10, Decimal('1.5'))
)
Order(joe, banana_cart, bulk_item_promo)  #3

<Order total: 30.00 due: 28.50

In [6]:
long_cart = tuple(LineItem(str(sku), 1, Decimal(1)) for sku in range(10))
Order(joe, long_cart, large_order_promo)

<Order total: 10.00 due: 9.30

In [7]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00

In [8]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)

In [9]:
Order(joe, long_cart, best_promo)

<Order total: 10.00 due: 9.30

In [10]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50

In [11]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90

## Decorator-Enhanced Strategy Pattern

In [ ]:
from decimal import Decimal
from typing import Callable

Promotion = Callable[[Order], Decimal]

promos: list[Promotion] = []  #1


def promotion(promo: Promotion) -> Promotion:  #2
    promos.append(promo)
    return promo

def best_promo(order: Order) -> Decimal:
    """Compute the best discount available"""
    return max(promo(order) for promo in promos)  #3

@promotion
def fidelity(order: Order) -> Decimal:
    """5% discount for customer with 1000 or more fidelity points"""
    if order.customer.fidelity >= 1000:
        return order.total() * Decimal('0.05')
    return Decimal(0)

@promotion  #4
def bulk_item(order: Order) -> Decimal:
    """10% discount for each LineItem with 20 or more units"""
    discount = Decimal(0)
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * Decimal('0.1')
    return discount

@promotion
def large_order(order: Order) -> Decimal:
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * Decimal('0.07')
    return Decimal(0)